In [ ]:
import subprocess
import os

os.chdir("./checker/checker")

# Command and arguments for the external program
command = ["main.exe"]
# command = ["ls"]

print("Tarted process")
# Open the program for interaction
with subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) as process:
    print("Trying to interact")
    # Send input to the program (if needed)
    user_input = "A1 \n"
    # stdout, stderr = process.communicate(input=user_input)
    # print(stdout)
    # print("Currently on stdout")
    # print(process.stdout.read())
    process.stdin.write("A1")
    print("Writing done")
    process.stdin.flush() 
    print("Flushed")

    for line in process.stdout.readline():
        print("Writing")
        print(line)

# # After you're done, ensure you close the process properly
# process.stdin.close()  # Close input stream when done
# process.stdout.close()  # Close output stream
# process.stderr.close()
# process.wait()  # Wait for the process to fully terminate
# # # Check the output and error
# # if process.returncode == 0:
# #     print("Program executed successfully.")
# #     print("Output:", process.stdout)
# # else:
# #     print("There was an error:")
# #     print(process.stderr)
# #     print(process.stdout)

Tarted process
Trying to interact
Writing done
Flushed


ValueError: I/O operation on closed file.

In [ ]:
import subprocess
import os

os.chdir("./checker/checker")

# Command and arguments for the external program
command = ["main.exe"]
# command = ["ls"]

print("Started process")
# Open the program for interaction
process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Function to read output until a specific prompt is found
def read_until_prompt(prompt):
    output = ""
    while True:
        line = process.stdout.readline()  # Read a line from the output
        output += line
        if prompt in line:  # Stop reading when the prompt is found
            break
    return output

# Function to send input and receive output
def interact_with_program(input_data, prompt):
    # Wait for the prompt to appear in the output
    output = read_until_prompt(prompt)
    print("Received prompt:", output)  # You can log or process the output as needed

    # Send input to the external program after the prompt
    process.stdin.write(input_data + "\n")
    process.stdin.flush()  # Ensure input is sent immediately
    
    return output


# First interaction: Wait for a prompt and then send the first input
prompt = "Pressed key:"
output1 = interact_with_program("10x10", prompt)
print("Output after first input:", output1)

# prompt = "Pressed key:"
# # Step 2: Second interaction (sending second input, possibly based on the first output)
# output2 = interact_with_program("5")
# print("Output after second input:", output2)

# # Step 3: Continue with further interactions as needed...
# output3 = interact_with_program("6")
# print("Output after third input:", output3)

# output3 = interact_with_program("0")
# print("Output after third input:", output3)

# After you're done, ensure you close the process properly
process.stdin.close()  # Close input stream when done
process.stdout.close()  # Close output stream
process.stderr.close()
process.wait()  # Wait for the process to fully terminate

Started process


In [1]:
import os 

os.chdir("./checker/checker")

In [ ]:
import subprocess
import time

# Start the main.exe process with buffering and stdout/stderr handling
process = subprocess.Popen(
    ["main.exe"],  # Replace with your actual path to main.exe
    stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True,
    universal_newlines=True, bufsize=1, shell = True  # Try to disable output buffering
)

# Function to check if the process is running and print its status
def check_process_status():
    return_code = process.poll()
    if return_code is None:
        print("Process is running")
    else:
        print(f"Process terminated with return code: {return_code}")

# Test if any output is being written to stdout or stderr
def read_initial_output(process, timeout=5):
    start_time = time.time()
    output = ""
    
    tm = time.time()
    while tm - start_time < timeout:
        print(f"I am in the first while for {tm - start_time}")
        line = process.stdout.readline()
        print("Finished reading stdout")
        if line:
            output += line
            print("STDOUT:", line.strip())
        else:
            print("I break here")
            break  # No more output
        tm = time.time()
    # Check for any errors
    print("I came here")
    error_output = process.stderr.readline()
    if error_output:
        print("STDERR:", error_output.strip())

    return output

# Step 1: Check if the process starts correctly
check_process_status()

# Step 2: Try reading initial output from stdout and stderr
initial_output = read_initial_output(process)

if not initial_output:
    print("No output from the program, it might be buffered or there might be an issue with the executable.")

# Step 3: If the program expects input, try sending a simple input (e.g., A1) to see if it responds
process.stdin.write("A1\n")
process.stdin.flush()

# Step 4: Try reading output again after sending input
follow_up_output = read_initial_output(process)

if not follow_up_output:
    print("No follow-up output after sending input. The program might not be responding as expected.")

# Step 5: Check process status again
check_process_status()

# Step 6: Close the process if it's still running
process.stdin.close()
process.stdout.close()
process.stderr.close()
process.wait()


Process is running
I am in the first while for 0.0


In [1]:
import os 

os.chdir("./checker/checker")

In [2]:
import subprocess
import threading
import time

# Start the main.exe process with buffering and stdout/stderr handling
process = subprocess.Popen(
    ["main.exe"],  # Replace with your actual path to main.exe
    stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True,
    universal_newlines=True, bufsize=1, shell=True
)

# Function to handle reading stdout asynchronously
def read_stdout(process):
    while True:
        output = process.stdout.readline()
        if output:
            print("STDOUT:", output.strip())
        else:
            break  # Process finished or no more output

# Function to handle reading stderr asynchronously
def read_stderr(process):
    while True:
        error = process.stderr.readline()
        if error:
            print("STDERR:", error.strip())
        else:
            break  # Process finished or no more errors

# Start threads to read stdout and stderr without blocking
stdout_thread = threading.Thread(target=read_stdout, args=(process,))
stderr_thread = threading.Thread(target=read_stderr, args=(process,))

stdout_thread.start()
stderr_thread.start()

# Step 1: Send input to the process (e.g., A1 dataset)
process.stdin.write("A1\n")
process.stdin.flush()

# Wait a bit to simulate waiting for the program to process the input
time.sleep(1)

# Step 2: Send another input (e.g., option 5 and 6)
process.stdin.write("5\n")
process.stdin.flush()

# Wait for output
time.sleep(1)

process.stdin.write("6\n")
process.stdin.flush()

# Wait for output
time.sleep(1)

process.stdin.write("0\n")
process.stdin.flush()

# Wait for both threads to finish (the process will terminate when no more output is available)
stdout_thread.join()
stderr_thread.join()

# Close the process properly
process.stdin.close()
process.stdout.close()
process.stderr.close()
process.wait()


STDOUT: Please enter your USED BATCH file Index:
STDOUT: ------------------- Optimization Checker -------------------
STDOUT: 
STDOUT: 
STDOUT: ------------------- Version - 23/05/2018 -------------------
STDOUT: 
STDOUT: 
STDOUT: ----------- Start files parsing -----------
STDOUT: 
STDOUT: --- Parsing Optimization Parameters file ---
STDOUT: Optimization Parameters file path: instances_checker/global_param.csv
STDOUT: nPlates: 100
STDOUT: widthPlates: 6000
STDOUT: heightPlates: 3210
STDOUT: min1Cut: 100
STDOUT: max1Cut: 3500
STDOUT: min2Cut: 100
STDOUT: minWaste: 20
STDOUT: --- Optimization Parameters file parsed successfully ---
STDOUT: 
STDOUT: --- Parsing Batch file ---
STDOUT: Batch file path: instances_checker/A1_batch.csv
STDOUT: Items: 5
STDOUT: Stacks: 1
STDOUT: --- Batch file parsed successfully ---
STDOUT: 
STDOUT: --- Parsing Solution file ---
STDOUT: Solution file path: instances_checker/A1_solution.csv
STDOUT: Plates: 1
STDOUT: Node 16: is a residual
STDOUT: Nodes: 17
STD

0